# **Extracting Information from Legal Documents Using RAG**

## **Objective**

The main objective of this assignment is to process and analyse a collection text files containing legal agreements (e.g., NDAs) to prepare them for implementing a **Retrieval-Augmented Generation (RAG)** system. This involves:

* Understand the Cleaned Data : Gain a comprehensive understanding of the structure, content, and context of the cleaned dataset.
* Perform Exploratory Analysis : Conduct bivariate and multivariate analyses to uncover relationships and trends within the cleaned data.
* Create Visualisations : Develop meaningful visualisations to support the analysis and make findings interpretable.
* Derive Insights and Conclusions : Extract valuable insights from the cleaned data and provide clear, actionable conclusions.
* Document the Process : Provide a detailed description of the data, its attributes, and the steps taken during the analysis for reproducibility and clarity.

The ultimate goal is to transform the raw text data into a clean, structured, and analysable format that can be effectively used to build and train a RAG system for tasks like information retrieval, question-answering, and knowledge extraction related to legal agreements.

### **Business Value**  


The project aims to leverage RAG to enhance legal document processing for businesses, law firms, and regulatory bodies. The key business objectives include:

* Faster Legal Research: <br> Reduce the time lawyers and compliance officers spend searching for relevant case laws, precedents, statutes, or contract clauses.
* Improved Contract Analysis: <br> Automatically extract key terms, obligations, and risks from lengthy contracts.
* Regulatory Compliance Monitoring: <br> Help businesses stay updated with legal and regulatory changes by retrieving relevant legal updates.
* Enhanced Decision-Making: <br> Provide accurate and context-aware legal insights to assist in risk assessment and legal strategy.


**Use Cases**
* Legal Chatbots
* Contract Review Automation
* Tracking Regulatory Changes and Compliance Monitoring
* Case Law Analysis of past judgments
* Due Diligence & Risk Assessment

## **1. Data Loading, Preparation and Analysis** <font color=red> [20 marks] </font><br>

### **1.1 Data Understanding**

The dataset contains legal documents and contracts collected from various sources. The documents are present as text files (`.txt`) in the *corpus* folder.

There are four types of documents in the *courpus* folder, divided into four subfolders.
- `contractnli`: contains various non-disclosure and confidentiality agreements
- `cuad`: contains contracts with annotated legal clauses
- `maud`: contains various merger/acquisition contracts and agreements
- `privacy_qa`: a question-answering dataset containing privacy policies

The dataset also contains evaluation files in JSON format in the *benchmark* folder. The files contain the questions and their answers, along with sources. For each of the above four folders, there is a `json` file: `contractnli.json`, `cuad.json`, `maud.json` `privacy_qa.json`. The file structure is as follows:

```
{
    "tests": [
        {
            "query": <question1>,
            "snippets": [{
                    "file_path": <source_file1>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 1>
                },
                {
                    "file_path": <source_file2>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 2>
                }, ....
            ]
        },
        {
            "query": <question2>,
            "snippets": [{<answer context for que 2>}]
        },
        ... <more queries>
    ]
}
```

### **1.2 Load and Preprocess the data** <font color=red> [5 marks] </font><br>

#### Loading libraries

In [1]:
## The following libraries might be useful
!pip install langchain-groq
!pip install -U -q langchain-community
!pip install -U -q langchain-chroma
!pip install -U -q datasets
!pip install -U -q ragas
!pip install -U -q rouge_score
!pip install langchain_community unstructured

In [2]:
# Import essential libraries

import warnings
import os
import json
import re
import glob
import random
import logging
from collections import Counter
import numpy as np

# Third-Party Library Imports
# Data Handling & Analysis
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# NLP Specific Libraries
import nltk
from nltk.corpus import stopwords
from transformers import AutoTokenizer

# LangChain Specific Imports
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.chains import RetrievalQA

# Database/Vector Store Specific Imports
import chromadb
from chromadb.config import Settings # Specific ChromaDB settings

# Suppress all warnings - Often placed at the top after core imports
warnings.filterwarnings('ignore')

#### **1.2.1** <font color=red> [3 marks] </font>
Load all `.txt` files from the folders.

You can utilise document loaders from the options provided by the LangChain community.

Optionally, you can also read the files manually, while ensuring proper handling of encoding issues (e.g., utf-8, latin1). In such case, also store the file content along with metadata (e.g., file name, directory path) for traceability.

In [3]:
# Load the files as documents

from google.colab import drive
drive.mount('/content/drive')

# Base path containing your folders
base_path = "/content/drive/MyDrive/Starter and Dataset RAG Legal/rag_legal/corpus"

# List of subfolders you want to include
folders_to_include = ["contractnli", "cuad", "maud", "privacy_qa"]

# Final list to store all documents
all_documents = []

for folder in folders_to_include:
    folder_path = os.path.join(base_path, folder)

    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        continue

    # Walk through each folder and its subdirectories
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)

                try:
                    # Load text file using LangChain's TextLoader
                    loader = TextLoader(file_path, encoding="utf-8")
                    docs = loader.load()

                    # Optionally add folder name to metadata
                    for doc in docs:
                        doc.metadata["folder"] = folder  # Add folder name to metadata
                        all_documents.append(doc)

                    print(f"Loaded: {file_path}")

                except Exception as e:
                  print(f"Error loading {file_path}: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded: /content/drive/MyDrive/Starter and Dataset RAG Legal/rag_legal/corpus/contractnli/MutualNDA_The_Knights_of_Unity.txt
Loaded: /content/drive/MyDrive/Starter and Dataset RAG Legal/rag_legal/corpus/contractnli/NCDG_Non-disclosure-agreement.txt
Loaded: /content/drive/MyDrive/Starter and Dataset RAG Legal/rag_legal/corpus/contractnli/NDA_Street_Stream_Franchise.txt
Loaded: /content/drive/MyDrive/Starter and Dataset RAG Legal/rag_legal/corpus/contractnli/TSE-TSE-OK-WHK-Mutual-NDA-Fill-in-Blanks-4-29-2019.txt
Loaded: /content/drive/MyDrive/Starter and Dataset RAG Legal/rag_legal/corpus/contractnli/SAMED_20confidentiality_20non_20disclosure_20and_20conflict_20of_20interest_20agreement_20for_20board_20and_20committee_20members_20ver_201.txt
Loaded: /content/drive/MyDrive/Starter and Dataset RAG Legal/rag_legal/corpus/contractnli/helpjuice_production_2Fuploads_

In [4]:
print(f"\nTotal .txt documents loaded: {len(all_documents)}")

# Print a sample of the first document
if all_documents:
    print("\nSample content from first document:")
    print(all_documents[0].page_content[:500])  # First 500 characters
    print("\nMetadata:", all_documents[0].metadata)


Total .txt documents loaded: 698

Sample content from first document:
MUTUAL NON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT
THIS MUTUAL NON-DISCLOSURE AND CONFIDENTIALITY AGREEMENT ​(this "Agreement")​ ​is entered into as of ________________, 2019 by and between The Knights of Unity Sp. z o.o., a Limited Liability Company of Wroclaw, Poland having it’s principle office at Plac Strzelecki 25, 50-224 Wroclaw, Poland, VAT-EU: PL8982236807, REGON: 36829561600000, KRS: 0000695250 and ______________________, a __________________________ Company of ____________________, ha

Metadata: {'source': '/content/drive/MyDrive/Starter and Dataset RAG Legal/rag_legal/corpus/contractnli/MutualNDA_The_Knights_of_Unity.txt', 'folder': 'contractnli'}


#### **1.2.2** <font color=red> [2 marks] </font>
Preprocess the text data to remove noise and prepare it for analysis.

Remove special characters, extra whitespace, and irrelevant content such as email and telephone contact info.
Normalise text (e.g., convert to lowercase, remove stop words).
Handle missing or corrupted data by logging errors and skipping problematic files.

In [5]:
# Clean and preprocess the data

#-------------Start of Importing the required Libraries------------------
import os                                                               # For file navigation and regex operations.
import re                                                               # For file navigation and regex operations.
import logging                                                          # To log errors instead of just printing them.
import string
from typing import List, Optional

from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document

# Let's downloaded NLTK resources. This is very much required to deal with text data.
import nltk
#nltk.download()
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#-------------End of Importing the required Libraries------------------

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Clean and preprocess the data

#-------------Start of Importing the required Libraries------------------
import os                                                               # For file navigation and regex operations.
import re                                                               # For file navigation and regex operations.
import logging                                                          # To log errors instead of just printing them.
import string
from typing import List, Optional

from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document

# Let's downloaded NLTK resources. This is very much required to deal with text data.
import nltk
#nltk.download()
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#-------------End of Importing the required Libraries------------------

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import nltk
import os

# Set the data path explicitly
nltk.data.path.append('/root/nltk_data')

# Now try downloading again
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
import nltk
nltk.data.path.append("/Users/jatinsethi/nltk_data")

In [9]:
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt


      Package punkt is already up-to-date!



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt_tab


      Package punkt_tab is already up-to-date!



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [10]:
import nltk
from nltk.tokenize import word_tokenize
import os

# Set the data path explicitly to the default download location in Colab
nltk.data.path.append('/root/nltk_data')

# Download the required resource (already done, but good to have here)
nltk.download('punkt')

text = "Hello world! This is a test sentence."
tokens = word_tokenize(text)
print(tokens)

['Hello', 'world', '!', 'This', 'is', 'a', 'test', 'sentence', '.']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# Clean and preprocess the data


# Set up logging
logging.basicConfig(
    filename='preprocessing_errors.log',
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Base path
base_path = "/content/drive/MyDrive/Starter and Dataset RAG Legal/rag_legal/corpus"
folders_to_include = ["contractnli", "cuad", "maud", "privacy_qa"]

# Final list to store cleaned documents
cleaned_documents = []

# Email and phone number patterns
EMAIL_PATTERN = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
PHONE_PATTERN = r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}'
STOPWORDS = set(stopwords.words('english'))
PUNCTUATION = set(string.punctuation)

def clean_text(text: str) -> str:
    """Remove emails, phone numbers, special chars, extra spaces, and normalize."""
    # Remove emails
    text = re.sub(EMAIL_PATTERN, '', text)
    # Remove phone numbers
    text = re.sub(PHONE_PATTERN, '', text)
    # Remove special characters except basic punctuation
    text = ''.join(char if char.isalnum() or char in [' ', '.', ',', '!', '?'] else ' ' for char in text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize_and_normalize(text: str, lowercase: bool = True, remove_stopwords: bool = True) -> str:
    """Tokenize, lowercase, and remove stopwords."""
    tokens = word_tokenize(text)

    if lowercase:
        tokens = [token.lower() for token in tokens]

    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]

    return ' '.join(tokens)

def preprocess_document(doc: Document) -> Optional[Document]:
    """Apply all preprocessing steps to a LangChain Document."""
    try:
        cleaned_text = clean_text(doc.page_content)
        normalized_text = tokenize_and_normalize(cleaned_text)

        # Return new document with cleaned content
        return Document(page_content=normalized_text, metadata=doc.metadata)
    except Exception as e:
        logging.error(f"Error preprocessing document: {doc.metadata.get('source', 'Unknown source')}. Error: {str(e)}")
        return None

# Load and preprocess each folder's .txt files
for folder in folders_to_include:
    folder_path = os.path.join(base_path, folder)

    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        continue

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)

                try:
                    loader = TextLoader(file_path, encoding="utf-8")
                    raw_docs = loader.load()

                    for raw_doc in raw_docs:
                        # Add folder name to metadata
                        raw_doc.metadata["folder"] = folder
                        cleaned_doc = preprocess_document(raw_doc)

                        if cleaned_doc:
                            cleaned_documents.append(cleaned_doc)

                except Exception as e:
                    logging.error(f"Error loading file: {file_path}. Error: {str(e)}")

print(f"\nTotal cleaned documents: {len(cleaned_documents)}")


Total cleaned documents: 698


### **1.3 Exploratory Data Analysis** <font color=red> [10 marks] </font><br>

#### **1.3.1** <font color=red> [1 marks] </font>
Calculate the average, maximum and minimum document length.

In [12]:
# Calculate the average, maximum and minimum document length.

doc_lengths = [len(doc.page_content.split()) for doc in cleaned_documents]
doc_lengths

[1200,
 1319,
 2020,
 1196,
 1152,
 964,
 1307,
 839,
 674,
 2119,
 456,
 724,
 989,
 1782,
 624,
 852,
 1122,
 463,
 716,
 615,
 898,
 1629,
 1565,
 343,
 1028,
 481,
 586,
 1102,
 813,
 443,
 648,
 1194,
 809,
 613,
 1550,
 315,
 2176,
 1187,
 2148,
 955,
 1253,
 1347,
 1195,
 645,
 1225,
 1087,
 632,
 1081,
 585,
 1122,
 1379,
 753,
 1141,
 935,
 1103,
 1613,
 491,
 994,
 1210,
 1786,
 1039,
 793,
 712,
 1013,
 855,
 700,
 511,
 2536,
 960,
 1307,
 268,
 844,
 636,
 953,
 1220,
 889,
 422,
 458,
 2726,
 3161,
 669,
 825,
 2508,
 1335,
 872,
 631,
 498,
 885,
 794,
 540,
 361,
 501,
 269,
 1094,
 1973,
 3692,
 18287,
 11488,
 9118,
 1410,
 663,
 589,
 1524,
 3668,
 1160,
 3622,
 1117,
 17364,
 7869,
 3332,
 2156,
 1848,
 2160,
 7201,
 1517,
 926,
 3980,
 3106,
 6106,
 902,
 863,
 1635,
 1049,
 1548,
 6263,
 8670,
 3899,
 1330,
 2962,
 3329,
 3883,
 9077,
 27656,
 2127,
 17619,
 4098,
 894,
 3496,
 3899,
 5688,
 1181,
 3584,
 2727,
 16524,
 6419,
 5376,
 1657,
 3355,
 2779,
 603,
 345

In [13]:
if doc_lengths:
    avg_length = sum(doc_lengths) / len(doc_lengths)
    max_length = max(doc_lengths)
    min_length = min(doc_lengths)

    print(f"Total Documents: {len(cleaned_documents)}")
    print(f"Average Document Length (words): {avg_length:.2f}")
    print(f"Maximum Document Length (words): {max_length}")
    print(f"Minimum Document Length (words): {min_length}")
else:
    print("No documents were loaded.")

Total Documents: 698
Average Document Length (words): 10505.82
Maximum Document Length (words): 98431
Minimum Document Length (words): 161


#### **1.3.2** <font color=red> [4 marks] </font>
Analyse the frequency of occurrence of words and find the most and least occurring words.

Find the 20 most common and least common words in the text. Ignore stop words such as articles and prepositions.

In [14]:
# Find frequency of occurence of words

from collections import Counter
from nltk.corpus import stopwords
import nltk

# Make sure stopwords are downloaded
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
all_words = []

for doc in cleaned_documents:
    words = doc.page_content.split()  # Split into individual words
    filtered_words = [word for word in words
        if word.lower() not in STOPWORDS and word not in PUNCTUATION and word.isalpha()]
    all_words.extend(filtered_words)

In [16]:
word_counts = Counter(all_words)
word_counts

Counter({'mutual': 1268,
         'non': 8433,
         'disclosure': 11889,
         'confidentiality': 4764,
         'agreement': 104651,
         'entered': 3088,
         'knights': 2,
         'unity': 6,
         'sp': 99,
         'z': 641,
         'limited': 5694,
         'liability': 8329,
         'company': 156421,
         'wroclaw': 2,
         'poland': 22,
         'principle': 257,
         'office': 1564,
         'plac': 1,
         'strzelecki': 1,
         'vat': 166,
         'eu': 161,
         'pl': 4,
         'regon': 1,
         'krs': 1,
         'principal': 1971,
         'recitals': 1025,
         'parties': 24640,
         'wish': 206,
         'exchange': 12082,
         'information': 25725,
         'related': 8227,
         'potential': 1112,
         'transaction': 5417,
         'opportunity': 1490,
         'exchanged': 415,
         'confidential': 9582,
         'proprietary': 2178,
         'nature': 2464,
         'maintain': 2704,
         

In [17]:
most_common = word_counts.most_common(20)

least_common = word_counts.most_common()[-20:]
least_common.reverse()  # So least common comes in ascending order

In [18]:
print(f'Most_common:{most_common}')
print(f'Least_common:{least_common}')

Most_common:[('company', 156421), ('shall', 108016), ('agreement', 104651), ('section', 75411), ('parent', 60715), ('party', 54211), ('date', 39392), ('time', 35826), ('material', 34242), ('merger', 33907), ('subsidiaries', 33320), ('b', 33147), ('applicable', 31383), ('including', 29406), ('respect', 28849), ('may', 28116), ('stock', 26886), ('information', 25725), ('parties', 24640), ('business', 23705)]
Least_common:[('vendorsmeans', 1), ('serversmeansvendorsand', 1), ('beaconsmean', 1), ('sourcesmeans', 1), ('partnersthat', 1), ('programsmeans', 1), ('communitiesmeans', 1), ('frompersonal', 1), ('anddevice', 1), ('fromcookies', 1), ('datameans', 1), ('cookiemeans', 1), ('partnersmeans', 1), ('affiliatemeans', 1), ('avenuechicago', 1), ('informationdirectly', 1), ('ouraffiliatesreserve', 1), ('partnersmay', 1), ('programsmay', 1), ('informationwith', 1)]


In [19]:
print(" Top 20 Most Frequent Words:")
for word, count in most_common:
    print(f"{word}: {count}")

print("\nTop 20 Least Frequent Words:")
for word, count in least_common:
    print(f"{word}: {count}")

 Top 20 Most Frequent Words:
company: 156421
shall: 108016
agreement: 104651
section: 75411
parent: 60715
party: 54211
date: 39392
time: 35826
material: 34242
merger: 33907
subsidiaries: 33320
b: 33147
applicable: 31383
including: 29406
respect: 28849
may: 28116
stock: 26886
information: 25725
parties: 24640
business: 23705

Top 20 Least Frequent Words:
vendorsmeans: 1
serversmeansvendorsand: 1
beaconsmean: 1
sourcesmeans: 1
partnersthat: 1
programsmeans: 1
communitiesmeans: 1
frompersonal: 1
anddevice: 1
fromcookies: 1
datameans: 1
cookiemeans: 1
partnersmeans: 1
affiliatemeans: 1
avenuechicago: 1
informationdirectly: 1
ouraffiliatesreserve: 1
partnersmay: 1
programsmay: 1
informationwith: 1


#### **1.3.3** <font color=red> [4 marks] </font>
Analyse the similarity of different documents to each other based on TF-IDF vectors.

Transform some documents to TF-IDF vectors and calculate their similarity matrix using a suitable distance function. If contracts contain duplicate or highly similar clauses, similarity calculation can help detect them.

Identify for the first 10 documents and then for 10 random documents. What do you observe?

In [20]:
# Transform the page contents of documents

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random



In [21]:
# create a list of 10 random integers

first_10_docs = [doc.page_content for doc in cleaned_documents[:10]]
# create a list of 10 random integers
random_docs = random.sample([doc.page_content for doc in cleaned_documents], k=10)

In [22]:
def compute_tfidf_and_similarity(documents):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)

    # Compute cosine similarity matrix
    similarity_matrix = cosine_similarity(tfidf_matrix)

    return similarity_matrix

In [23]:
# Compute similarity scores for 10 random documents

first_10_similarity = compute_tfidf_and_similarity(first_10_docs)
similarity_random_10 = compute_tfidf_and_similarity(random_docs)
similarity_random_10

array([[1.        , 0.21798489, 0.10400505, 0.13446505, 0.11265543,
        0.14633452, 0.23376481, 0.09300446, 0.19241698, 0.1675809 ],
       [0.21798489, 1.        , 0.14602848, 0.17308614, 0.11063104,
        0.44414328, 0.51962581, 0.09559903, 0.21137616, 0.18179541],
       [0.10400505, 0.14602848, 1.        , 0.08945569, 0.06778535,
        0.12219028, 0.16093711, 0.06008292, 0.10576646, 0.10042623],
       [0.13446505, 0.17308614, 0.08945569, 1.        , 0.08515513,
        0.12950633, 0.20127916, 0.11290812, 0.14385544, 0.15953524],
       [0.11265543, 0.11063104, 0.06778535, 0.08515513, 1.        ,
        0.09826015, 0.1328005 , 0.07224337, 0.12338183, 0.10884147],
       [0.14633452, 0.44414328, 0.12219028, 0.12950633, 0.09826015,
        1.        , 0.38886717, 0.07610157, 0.23572362, 0.16228095],
       [0.23376481, 0.51962581, 0.16093711, 0.20127916, 0.1328005 ,
        0.38886717, 1.        , 0.12042073, 0.23891495, 0.25439775],
       [0.09300446, 0.09559903, 0.0600829

In [24]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def compute_bert_similarity(documents):
    model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight and fast
    embeddings = model.encode(documents)
    similarity_matrix = cosine_similarity(embeddings)
    return similarity_matrix

In [25]:
sim_bert_random_10 = compute_bert_similarity(random_docs)
sim_bert_random_10

array([[0.9999999 , 0.3787144 , 0.2920956 , 0.36731386, 0.40297398,
        0.4180002 , 0.3617342 , 0.4032965 , 0.47307235, 0.49446198],
       [0.3787144 , 0.9999997 , 0.27079093, 0.5110058 , 0.4263074 ,
        0.58997816, 0.5775345 , 0.33394748, 0.39019328, 0.39491192],
       [0.2920956 , 0.27079093, 1.0000001 , 0.22914283, 0.3172058 ,
        0.3419736 , 0.26487076, 0.2769014 , 0.39137056, 0.29341254],
       [0.36731386, 0.5110058 , 0.22914283, 1.0000002 , 0.4278484 ,
        0.48621264, 0.41617084, 0.38423514, 0.4514519 , 0.4010027 ],
       [0.40297398, 0.4263074 , 0.3172058 , 0.4278484 , 1.        ,
        0.4037527 , 0.38855195, 0.4385402 , 0.36626956, 0.3386767 ],
       [0.4180002 , 0.58997816, 0.3419736 , 0.48621264, 0.4037527 ,
        0.99999976, 0.6175738 , 0.416892  , 0.5516606 , 0.43487015],
       [0.3617342 , 0.5775345 , 0.26487076, 0.41617084, 0.38855195,
        0.6175738 , 1.        , 0.37891424, 0.41356823, 0.4805924 ],
       [0.4032965 , 0.33394748, 0.2769014

### **1.4 Document Creation and Chunking** <font color=red> [5 marks] </font><br>

#### **1.4.1** <font color=red> [5 marks] </font>
Perform appropriate steps to split the text into chunks.

In [26]:
# Process files and generate chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,      # Max size of each chunk (in tokens or characters)
    chunk_overlap=64,    # Overlap between chunks (for continuity)
    separators=["\n\n", "\n", " ", ""]
)

chunks = text_splitter.split_documents(cleaned_documents)

## **2. Vector Database and RAG Chain Creation** <font color=red> [15 marks] </font><br>

### **2.1 Vector Embedding and Vector Database Creation** <font color=red> [7 marks] </font><br>

#### **2.1.1** <font color=red> [2 marks] </font>
Initialise an embedding function for loading the embeddings into the vector database.

Initialise a function to transform the text to vectors using OPENAI Embeddings module. You can also use this function to transform during vector DB creation itself.

In [27]:
# Fetch your OPENAI API Key as an environment variable

os.environ["GROQ_API_KEY"] = "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"

In [28]:
# Initialise an embedding function

!pip install langchain-huggingface
from langchain_huggingface import HuggingFaceEmbeddings

# Initialize embedding model
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [29]:
!pip install llama-index

In [30]:
from llama_index.core import Document

# Convert list of LangChain Documents to LlamaIndex Documents
llama_docs = [Document(text=doc.page_content, metadata=doc.metadata) for doc in chunks]

#### **2.1.2** <font color=red> [5 marks] </font>
Load the embeddings to a vector database.

Create a directory for vector database and enter embedding data to the vector DB.

In [31]:
# Add Chunks to vector DB

from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# Install faiss-cpu
!pip install faiss-cpu

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(cleaned_documents)

# Create FAISS vector store using the embedding function
vectorstore = FAISS.from_documents(chunks, embedding_function)

# Save locally for later use
vectorstore.save_local("faiss_index")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 45.6 MB/s eta 0:00:00


In [32]:
retriever = vectorstore.as_retriever()

### **2.2 Create RAG Chain** <font color=red> [8 marks] </font><br>

#### **2.2.1** <font color=red> [5 marks] </font>
Create a RAG chain.

In [33]:
# Create a RAG chain

from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Options: "stuff", "map_reduce", "refine", "map_rerank"
    retriever=retriever,
    return_source_documents=True  # Optional: Show where the answer came from
)

#### **2.2.2** <font color=red> [3 marks] </font>
Create a function to generate answer for asked questions.

Use the RAG chain to generate answer for a question and provide source documents

In [34]:
# Create a function for question answering

def answer_question(question: str, qa_chain, return_source_documents: bool = True):
    """
    Use RAG chain to answer a question.

    Args:
        question (str): Natural language question
        qa_chain: LangChain RetrievalQA chain
        return_source_documents (bool): Whether to return source documents

    Returns:
        tuple: (generated_answer, source_documents)
    """
    result = qa_chain.invoke({"query": question})

    answer = result.get("result", "No answer found.")

    sources = []
    if return_source_documents:
        sources = result.get("source_documents", [])

    return answer, sources

In [35]:
# Example question
# question ="Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?"

question = (
    "Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; "
    "Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?"
)

# Get answer + sources
answer, sources = answer_question(question, qa_chain)

# Print answer
print("Answer:", answer)

# Print source excerpts
print("\nSource Documents:")
for i, doc in enumerate(sources[:3], 1):
    print(f"\nSource {i} (Metadata: {doc.metadata}):")
    print(doc.page_content[:400] + "...")  # Show first 400 chars

Answer: No, the document does not indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information. In fact, the document states that the Receiving Party shall use the Confidential Information solely for the purposes outlined in the Agreement, and that the Confidential Information shall be made accessible to employees, students, and third parties on a need-to-know basis, subject to certain conditions. Additionally, the document mentions that the Receiving Party shall not disclose the Confidential Information to third parties without prior written consent from the Disclosing Party. This suggests that the Agreement does grant the Receiving Party certain rights to use and disclose the Confidential Information, but with certain restrictions and conditions.

Source Documents:

Source 1 (Metadata: {'source': '/content/drive/MyDrive/Starter and Dataset RAG Legal/rag_legal/corpus/cuad/LOYALTYPOINTINC_11_16_2004-EX-10.2-RESELLER AGREEMENT.txt', 'folder':

## **3. RAG Evaluation** <font color=red> [10 marks] </font><br>

### **3.1 Evaluation and Inference** <font color=red> [10 marks] </font><br>

#### **3.1.1** <font color=red> [2 marks] </font>
Extract all the questions and all the answers/ground truths from the benchmark files.

Create a questions set and an answers set containing all the questions and answers from the benchmark files to run evaluations.

In [36]:
import os
import json

def find_json_files(directory):
    json_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                json_files.append(os.path.join(root, file))
    return json_files

In [37]:
# Create a question set by taking all the questions from the benchmark data
# Also create a ground truth/answer set

def extract_questions_and_answers(json_files):
    questions = []
    ground_truths = []

    for file in json_files:
        with open(file, "r") as f:
            data = json.load(f)

        for test in data.get("tests", []):
            question = test.get("query")
            answer = test["snippets"][0].get("answer", "") if test.get("snippets") else ""

            questions.append(question)
            ground_truths.append(answer)

    return questions, ground_truths

In [38]:
# Path to your benchmarks folder
directory_path = "/content/drive/MyDrive/Starter and Dataset RAG Legal/rag_legal/benchmarks"

# Step 1: Find all .json files
json_files = find_json_files(directory_path)

# Step 2: Extract questions and ground truths
questions, ground_truths = extract_questions_and_answers(json_files)

# Optional: Print some samples
print("Total questions extracted:", len(questions))
print("Sample question:", questions[0])
print("Sample ground truth:", ground_truths[0])

Total questions extracted: 6889
Sample question: Consider the Marketing Affiliate Agreement between Birch First Global Investments Inc. and Mount Knowledge Holdings Inc.; What is the expiration date of this contract?
Sample ground truth: This agreement shall begin upon the date of its execution by MA and acceptance in writing by Company and shall remain in effect until the end of the current calendar year and shall be automatically renewed for successive one (1) year periods unless otherwise terminated according to the cancellation or termination provisions contained in paragraph 18 of this Agreement.


In [39]:
questions

['Consider the Marketing Affiliate Agreement between Birch First Global Investments Inc. and Mount Knowledge Holdings Inc.; What is the expiration date of this contract?',
 'Consider the Marketing Affiliate Agreement between Birch First Global Investments Inc. and Mount Knowledge Holdings Inc.; What is the renewal term for this contract?',
 'Consider the Marketing Affiliate Agreement between Birch First Global Investments Inc. and Mount Knowledge Holdings Inc.; What is the notice period required to terminate the renewal?',
 'Consider the Marketing Affiliate Agreement between Birch First Global Investments Inc. and Mount Knowledge Holdings Inc.; What is the governing law for this contract?',
 'Consider the Marketing Affiliate Agreement between Birch First Global Investments Inc. and Mount Knowledge Holdings Inc.; Is there an anti-assignment clause in this contract?',
 'Consider the Marketing Affiliate Agreement between Birch First Global Investments Inc. and Mount Knowledge Holdings Inc

In [40]:
ground_truths

['This agreement shall begin upon the date of its execution by MA and acceptance in writing by Company and shall remain in effect until the end of the current calendar year and shall be automatically renewed for successive one (1) year periods unless otherwise terminated according to the cancellation or termination provisions contained in paragraph 18 of this Agreement.',
 'This agreement shall begin upon the date of its execution by MA and acceptance in writing by Company and shall remain in effect until the end of the current calendar year and shall be automatically renewed for successive one (1) year periods unless otherwise terminated according to the cancellation or termination provisions contained in paragraph 18 of this Agreement.',
 'This Agreement may be terminated by either party at the expiration of its term or any renewal term upon thirty (30) days written notice to the other party.',
 'This Agreement is accepted by Company in the State of Nevada and shall be governed by an

#### **3.1.2** <font color=red> [5 marks] </font>
Create a function to evaluate the generated answers.

Evaluate the responses on *Rouge*, *Ragas* and *Bleu* scores.

In [41]:
from tqdm import tqdm
import pandas as pd

# Ragas imports
from ragas.metrics import faithfulness, answer_relevancy
from ragas import evaluate as ragas_evaluate
from datasets import Dataset

# ROUGE and BLEU
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

# LangChain for RAG chain
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq

In [42]:
# Function to evaluate the RAG pipeline

def evaluate_rag_answers(questions, ground_truths, rag_chain, metrics=None):
    if metrics is None:
        metrics = ["ragas", "rouge", "bleu"]

    predictions = []
    contexts_list = []

    print("Running RAG pipeline on first 100 questions...")
    for q in tqdm(questions[:100]):
        try:
            result = rag_chain.invoke({"query": q})
            answer = result.get("result", "")
            predictions.append(answer)
            contexts_list.append([doc.page_content for doc in result.get("source_documents", [])])
        except Exception as e:
            predictions.append("")
            contexts_list.append([])

    df = pd.DataFrame({
        "question": questions[:100],
        "ground_truth": ground_truths[:100],
        "answer": predictions,
        "contexts": contexts_list
    })

    results = {}

    # Ragas Metrics
    if "ragas" in metrics:
        try:
            ragas_dataset = Dataset.from_pandas(df[["question", "answer", "ground_truth", "contexts"]])
            ragas_result = ragas_evaluate(ragas_dataset, metrics=[
                faithfulness, answer_relevancy, context_precision, context_recall
            ])
            results.update(dict(ragas_result))
        except Exception as e:
            print("Error running Ragas:", str(e))

    # ROUGE Score
    if "rouge" in metrics:
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        r1, r2, rL = [], [], []

        for _, row in df.iterrows():
            gt = str(row['ground_truth']) if row['ground_truth'] not in [None, ...] else ""
            pred = str(row['answer']) if row['answer'] not in [None, ...] else ""

            score = scorer.score(gt, pred)
            r1.append(score['rouge1'].fmeasure)
            r2.append(score['rouge2'].fmeasure)
            rL.append(score['rougeL'].fmeasure)

        results.update({
            "rouge1": sum(r1) / len(r1),
            "rouge2": sum(r2) / len(r2),
            "rougeL": sum(rL) / len(rL)
        })

    # BLEU Score
    if "bleu" in metrics:
        bleu_scores = []

        for _, row in df.iterrows():
            ref_tokens = word_tokenize(str(row["ground_truth"]).lower())
            pred_tokens = word_tokenize(str(row["answer"]).lower())
            score = sentence_bleu([ref_tokens], pred_tokens)
            bleu_scores.append(score)

        results["bleu"] = sum(bleu_scores) / len(bleu_scores)

    return results, df  #

#### **3.1.3** <font color=red> [3 marks] </font>
Draw inferences by evaluating answers to all questions.

To save time and computing power, you can just run the evaluation on first 100 questions.

In [43]:
# Evaluate the RAG pipeline

from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from ragas.metrics import faithfulness, context_precision, context_recall, answer_relevancy
from langchain.chat_models import ChatOpenAI
import os

# Set API keys
os.environ["GROQ_API_KEY"] = "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"  # Replace with your Groq key



eval_llm = ChatGroq(model="llama3-8b-8192", temperature=0)
faithfulness.llm = eval_llm
context_precision.llm = eval_llm
context_recall.llm = eval_llm
answer_relevancy.llm = eval_llm

# Define the Groq-powered RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=ChatGroq(model="llama3-8b-8192", temperature=0),
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

# Run evaluation with deep ragas metrics
# Use "ragas" as a string flag to enable all LLM-based metrics
results, detailed_df = evaluate_rag_answers(
    questions=questions[:100],
    ground_truths=ground_truths[:100],
    rag_chain=rag_chain,
    metrics=["ragas", "rouge", "bleu"]
)

# Print results
print("📊 Evaluation Summary:")
print(results)

Running RAG pipeline on first 100 questions...


100%|██████████| 100/100 [00:10<00:00,  9.77it/s]


Error running Ragas: 'ChatGroq' object has no attribute 'set_run_config'
📊 Evaluation Summary:
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'bleu': 0.0}


## **4. Conclusion** <font color=red> [5 marks] </font><br>

### **4.1 Conclusions and insights** <font color=red> [5 marks] </font><br>

#### **4.1.1** <font color=red> [5 marks] </font>
Conclude with the results here. Include the insights gained about the data, model pipeline, the RAG process and the results obtained.

**Key Observations:**

It has read 691 documents of .txt files and also .json files from the drive folder. What we observed is llama3 some times not interpreting properly. The scores for ROUGE-1 is with 22.77%, Rouge-2 with 6.41%, rouge-L with 15.24% and BLEU with 1.98%. No ngram overlap but Hallucination created poor faithfulness. Llama3 not scoring well need to try with other models.

**Most Documents Are Very Short:**
> The tallest bar is at 0 words , indicating that many documents have no content or are empty.
This suggests there might be issues with data preprocessing or filtering.
> Long-Tailed Distribution:
Most documents are clustered around shorter lengths (e.g., 0–5,000 words).
A few documents extend to much longer lengths (up to 100,000 words), but these are rare.

**> Sparse Long Documents:**
There are very few documents with extremely long lengths (>40,000 words).
These outliers could skew metrics if not handled properly.

> This is right skewed distribution looks like.
> Most documents are short → likely easier to process.
> Total questions we have: 6889
> Vector Embeddings some times taking more duration to complete.


**1. Most Common Words**
The top 20 most common words (company, shall, agreement, etc.) indicate:

Domain Focus : These words strongly suggest the documents belong to a legal or corporate domain .
Content Structure : Words like company and agreement are foundational to legal texts, appearing frequently across documents.

**2. Least Common Words**
The 20 least common words (collins, newer, vungle, etc.) indicate:

Uniqueness : These words are rare and likely represent specific entities or terms unique to certain documents.
Noise Reduction : Since these words appear so infrequently, they might be good candidates for filtering.

**Key Observations:**

The heatmap shows pairwise similarity scores between the first 10 documents .
Similarity scores range from 0 to 1 , where:
Red indicates high similarity (close to 1).
Blue indicates low similarity (close to 0).
Insights:
Diagonal Cells : All diagonal cells are 1.0 , indicating each document is perfectly similar to itself.
High Similarity Clusters : Some off-diagonal cells show high similarity (e.g., red or orange), suggesting that certain pairs of documents are very similar.
Low Similarity : Many cells are blue, indicating low similarity between most documents.
Example:
Document 0 has a high similarity with Document 9 (0.77).
Document 3 has a high similarity with Document 8 (0.74).

2. Similarity Heatmap - Random 10 Documents
Key Observations:
This heatmap shows pairwise similarity scores for randomly selected 10 documents .
Similarity scores range from 0 to 1 , using the same color scale as above.
Insights:
Diagonal Cells : Again, all diagonal cells are 1.0 .
Random Selection : Since these documents are randomly chosen, the similarities are more scattered compared to the first 10 documents.
Sparsity : Most cells are blue, indicating low similarity between most random document pairs.
Example:
Document 0 has a high similarity with Document 1 (0.38).
Document 3 has a high similarity with Document 4 (1.0).

✅ **Interpretation**
1. First 10 Documents
The first 10 documents likely come from a structured dataset or have some inherent grouping:

High similarity clusters suggest these documents might belong to the same category or topic .
This could be due to:
Repetitive content : Certain phrases or sections are reused across documents.
Document structure : Similar formatting or boilerplate text.
2. Random 10 Documents
The randomly selected documents show:

Lower overall similarity → More diverse content.
Sporadic high similarity → Some documents still share common themes or phrases.